In [54]:
import tweepy
import pandas as pd 

text_file = open("twitter_OAuth.txt", "r")
C_KEY, C_SECRET, A_TOKEN, A_TOKEN_SECRET = text_file.read().split('\n')

auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN, A_TOKEN_SECRET)

api = tweepy.API(auth)



In [58]:
userId = 'joshuawongcf'
# Get the User object for twitter...
user = api.get_user(userId)

print('Username ' + user.screen_name)

Username joshuawongcf


In [59]:
###################################################
# Get all id of people the the user (Joshua) follows
# Method 1:
#friends = api.friends_ids(userId)

#Method 2:
users = tweepy.Cursor(api.friends, screen_name=user.screen_name,count=200).items()


In [61]:
screennamelst = list()
namelst0 = list()
userIdlst = list()
locationlst = list()
friends_countlst, listed_countlst, favourites_countlst, statuses_countlst, verifiedlst= [],[], [], [], []
for u in users:   
    namelst0.append(u.name)
    screennamelst.append(u.screen_name)
    userIdlst.append(u.id)
    locationlst.append(u.location)
    friends_countlst.append(u.friends_count)
    listed_countlst.append(u.listed_count)
    favourites_countlst.append(u.favourites_count)
    statuses_countlst.append(u.statuses_count)
    verifiedlst.append(u.verified)

print(len(screennamelst))
print(len(namelst0))
print(len(userIdlst))

1119
1119
1119


In [63]:
joshua_following = pd.DataFrame({'screen_name':screennamelst, 'user_name':namelst0, 'user_id':userIdlst, \
                    'location':locationlst, 'friends_count': friends_countlst, 'listed_count':listed_countlst, \
                    'favourites_count':favourites_countlst, \
                    'statuses_count':statuses_countlst, 'verified':verifiedlst})
joshua_following.to_csv('following.csv', sep=',')

print("csv have been created")

csv have been created


In [65]:
#######################################################
# read csv
# use a double for loop to check them is they have mutual friendship
# if yes, add an edge to G
import networkx as nx
data = pd.read_csv("following.csv") 
screennamelst_csv = data["screen_name"]
namelst_csv = data["user_name"]
idlst_csv = data["user_id"]

G = nx.Graph()

res =  [False for i in range(1119)] 

In [95]:
edgelst = list()

#for each screenname in csv file
index = 0
for friend_id in idlst_csv:
    if res[index] == False:
        #get friends list of this twitter user.
        friend_of_friend_ids = tweepy.Cursor(api.friends_ids, id=friend_id,count=200).items()
        fd_of_fd_lst = list()
        for u in friend_of_friend_ids:   
            fd_of_fd_lst.append(u)
        fd_of_fd_df = pd.DataFrame({'id':fd_of_fd_lst}) 
        fd_of_fd_df.to_csv('following/following_'+ str(friend_id)+'.csv', sep=',')
        res[index] = True
    index= index +1

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [ ]:
#Compare 
for idx, friend_of_friend_id in enumerate(friend_of_friend_ids):
    if friend_of_friend_id in idlst_csv:
        edgelst.add([ userId, idlst_csv[idx]])